# Causal Inference
# School of Information, University of Michigan 
## Week 2

### Resources:
- Course Manual, which can be found in Coursera
- ["Nike Says Its \$250 Running Shoes Will Make You Run Much Faster. What if That’s Actually True?"](assets/NYT_Nike.pdf)

## Part 1

### Background
Nike claims that its $250 running shoes called “Vaporfly” will make you run much faster!

### Data 

The data file “lecture2_match_reg.csv” contains 5 variables for 24,699 runners that qualified for and ran the same marathon. Below are the descriptions of each variable in the data: 

- \`age\`: age of runner (min value: 18, max value: 55)
- \`male\`: dummy variable for gender; equal to 0 if female, 1 if male 
- \`marathoner_type\`: 
    - “seasoned” if runner has at least 3 prior completed marathons, 
    - “enthusiastic” if runner has completed 1 or 2 prior completed marathons, 
    - “first_timer” if this is a runner’s first time running a marathon 
- \`vaporfly\`: 1 if a runner’s racing shoe is Nike Vaporfly, 0 otherwise 
- \`race_time\`: marathon completion time in seconds

In [1]:
#Import Statements. Run this cell.
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from causalinference import CausalModel

In [2]:
#Uploading data for assignment. Run this cell.
data_marathon = pd.read_csv('assets/lecture2_match_reg.csv')

#Uncomment below to see the first five lines of the dataframe.
data_marathon.head()

,age,marathoner_type,vaporfly,race_time,male
0,41,enthusiastic,1,11755.176,1
1,42,enthusiastic,1,14980.950,0
2,39,enthusiastic,0,12342.542,1
3,29,enthusiastic,0,13142.107,1
4,34,enthusiastic,1,13255.874,0


## Questions

Using data on race times, we want to explore the accuracy of this claim using marathon runners data. To be more specific, Nike’s claim is that runners run 4% faster with Vaporfly shoes.

**Note**: You can refer to the manual for the methods we use in the assignment if you need to. 

**Use the data_marathon dataframe uploaded above to answer the questions below unless otherwise specified.**

**1.** In order to be able to interpret our results in the same format (i.e. percentage change), we want to conduct our analysis over the (natural) log of race times. Transform the original race_time variable by taking its natural log, call the new variable `ln_race_time`, and add it to the data_marathon dataframe. We will use *ln_race_time* as our outcome variable throughout the entire assignment. (1 pt)

**Tip**: Use the `.head()` or `.sample()` methods to check your work.

In [3]:
# YOUR CODE HERE

#Take natural log of race_time and add it to data_marathon:
data_marathon['ln_race_time'] = np.log(data_marathon['race_time'])

data_marathon.head()

#raise NotImplementedError()

,age,marathoner_type,vaporfly,race_time,male,ln_race_time
0,41,enthusiastic,1,11755.176,1,9.372049
1,42,enthusiastic,1,14980.950,0,9.614535
2,39,enthusiastic,0,12342.542,1,9.420807
3,29,enthusiastic,0,13142.107,1,9.483577
4,34,enthusiastic,1,13255.874,0,9.492196


In [4]:
# Hidden Tests, checking the value of ln_race_time.

**2.** Compute the means of ln_race_time for runners who wore Vaporfly and for those who did not. What is the difference between the means across those two groups? Assign the value to the variable `mean_diff1_2`. Ensure that the response is correct to at least 4 decimal points and that its data type is float. (1 pt)

In [5]:
# YOUR CODE HERE
#Mean difference between ln_race_time, wearing vapourfly Vs not and rounded up to 4 decimal points: 
mean_diff1_2 = (data_marathon['ln_race_time'][data_marathon['vaporfly']==1].mean()- data_marathon['ln_race_time'][data_marathon['vaporfly']==0].mean()).round(4)

mean_diff1_2

#Why is this negaive. It could be because it took shorter race time using vaporfly shoes. 
#raise NotImplementedError()

-0.064

In [6]:
# Hidden Tests, checking the value of mean_diff1_2.

**3.** Suppose the only thing that matters for race time is age. We want to estimate average treatment effects (ATE) of wearing Nike Vaporfly shoes, using nearest neighbor matching on variable *age* with respect to Euclidean distance. Use the `CausalModel` module and assign the ATE to the variable `ate1_3`. Ensure that the data type of *ate1_3* is float. (Round to three decimal places.) (2 pts)

**Use the data_sample1_3 dataframe for your response below (Question 3); data_sample1_3 is a subsample of our marathon data, which will save computational time for this question.**

In [7]:
data_sample1_3 = data_marathon.sample(n = 2000, random_state = 123)

In [8]:
# YOUR CODE HERE
data_sample1_3.head()

#Store variable age as array:
age_array = np.array(data_sample1_3['age'])

#Use Causal Model to assess the impact of age variable on people wearning vaporfly shoes or not on the racetime: 
m = CausalModel(Y = data_sample1_3['ln_race_time'].values,  #Outcome variable race time
                D= data_sample1_3['vaporfly'].values,       #Treatment variable 'vaporfly'
                X = age_array )                             #Using 'age' as covariate array

#Get nearest neighbour matching estimates of ATE, ATC, ATT   
m.est_via_matching()

print(m.estimates)

#Filter out ATE from m.estimates upto 3 decimal places: 
ate1_3 = m.estimates['matching']['ate'].round(3)

#raise NotImplementedError()


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.037      0.006     -6.307      0.000     -0.049     -0.026
           ATC     -0.035      0.006     -5.862      0.000     -0.047     -0.024
           ATT     -0.039      0.006     -6.436      0.000     -0.051     -0.027



In [9]:
print(f" ATE using Nearest Neighbour: {ate1_3}")

 ATE using Nearest Neighbour: -0.037


In [10]:
# Hidden Tests, checking the value of ate1_3.

**4.** We want to conduct propensity score matching to estimate the ATE of running with Nike Vaporfly shoes.

**Note: We are back to using the original dataframe data_marathon.**

**4a.** Create binary variables called `seasoned` and `enthusiastic` that are equal to 1 for the corresponding *marathoner_type* values and 0 otherwise. Add these variables to the data_marathon dataframe. (1 pt)

**Tip**: Use the `np.where()` method to create the binary variables.

In [11]:
# YOUR CODE HERE
#Convert marathoner_type in to binary values (seasonsed|enthusiastic 1, otherwise 0)

data_marathon['seasoned'] = np.where(data_marathon['marathoner_type']=='seasoned', 1, 0)
data_marathon['enthusiastic'] = np.where(data_marathon['marathoner_type']=='enthusiastic', 1, 0)

#See data:
data_marathon.head()
#raise NotImplementedError()

,age,marathoner_type,vaporfly,race_time,male,ln_race_time,seasoned,enthusiastic
0,41,enthusiastic,1,11755.176,1,9.372049,0,1
1,42,enthusiastic,1,14980.950,0,9.614535,0,1
2,39,enthusiastic,0,12342.542,1,9.420807,0,1
3,29,enthusiastic,0,13142.107,1,9.483577,0,1
4,34,enthusiastic,1,13255.874,0,9.492196,0,1


In [12]:
# Hidden Tests, checking the values of seasoned and enthusiastic.

**4b.** Use propensity score matching with a logit model to estimate the ATE of wearing Nike Vaporfly shoes. Use the variables *age*, *male*, *seasoned*, *enthusiastic* and the methods `est_propensity()` and `est_via_matching()` from the CausalModel module. Assign the ATE to the variable `ate1_4b` and ensure that its data type is float. (Round to three decimal places.) (2 pts)

**Use the data_sample1_4b dataframe for your response below (Question 3); data_sample1_3 is a subsample of our marathon data, which will save computational time for this question.**

In [13]:
data_sample1_4b = data_marathon.sample(n = 2000, random_state = 123)
data_sample1_4b.head()

,age,marathoner_type,vaporfly,race_time,male,ln_race_time,seasoned,enthusiastic
17708,32,first_timer,0,13415.146,0,9.504140,0,0
10157,34,first_timer,0,16504.877,0,9.711411,0,0
21933,35,first_timer,1,14836.316,0,9.604833,0,0
13741,37,first_timer,0,17957.191,0,9.795746,0,0
4524,26,seasoned,1,14254.088,1,9.564799,1,0


In [14]:
# YOUR CODE HERE

#Store age, gender, seasoned and enthusiast variable as covariate array: 
variables_array = np.array(data_sample1_4b[['age', 'male', 'seasoned', 'enthusiastic']])
#Get shape of array:
print(variables_array.shape)

#Now, let's use casual model on variables_array for people wearing vaporfly or not: 
m_2 = CausalModel(Y = data_sample1_4b['ln_race_time'].values, #Outcome variable race time
                  D = data_sample1_4b['vaporfly'].values,     #Wearning vaporfly or not
                  X = variables_array)                                                               #Covariate Array

#Estimate Propensity Score to get ATE, ATC, ATT: 
m_2.est_propensity()
m_2.est_via_matching()

print(m_2.estimates)

#Filter out ATE from m_2.estimates: 
ate1_4b = m_2.estimates['matching']['ate'].round(3)

#raise NotImplementedError()

(2000, 4)

Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.037      0.006     -6.762      0.000     -0.048     -0.027
           ATC     -0.036      0.006     -6.304      0.000     -0.047     -0.025
           ATT     -0.039      0.006     -6.433      0.000     -0.051     -0.027



In [15]:
print(f" ATE using Propensity Score: {ate1_4b}")

 ATE using Propensity Score: -0.037


In [16]:
#Hidden Tests, checking the value of ate1_4b.

## Part 2

> “Although regression is a many-splendored thing, we think of it as an automated matchmaker. Specifically, regression estimates are weighted averages of multiple matched comparisons of the sort constructed for the groups in our stylized matching matrix.” (Mastering Metrics, p. 49)

We now want to use controlled regression to conduct our analysis.

**1.** Using robust standard errors, regress *ln_race_time* on variables *vaporfly*, *male*, *seasoned*, and *enthusiastic*. Assign the results (using the `.get_robustcov_results()` method) to the variable `ols_robust2_1`.  (2 pts)

In [17]:
# YOUR CODE HERE

#Regress ln_race_time on various variables:
m_3  = smf.ols("ln_race_time~vaporfly+male+seasoned+enthusiastic", data = data_marathon)

#Fit m_3 (old model):
result = m_3.fit()

#Print model summary: 
print(result.summary())
#Here we noticed variance in standard error between (vaporfly, seasoned, enthusiastic), hence we will use 
#robust standrad error to get the result: 

#Assign object result of Robust standard error to ols_robust2_1, which will give us an object: 
ols_robust2_1 = result.get_robustcov_results(cov_type= 'HC1')
ols_robust2_1

#raise NotImplementedError()

                            OLS Regression Results                            
Dep. Variable:           ln_race_time   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                     2691.
Date:                Tue, 16 Feb 2021   Prob (F-statistic):               0.00
Time:                        22:48:47   Log-Likelihood:                 17850.
No. Observations:               24699   AIC:                        -3.569e+04
Df Residuals:                   24694   BIC:                        -3.565e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        9.6532      0.001   7501.940   

In [18]:
print(f"Summary of Robust Standard Error: {ols_robust2_1.summary()}")

Summary of Robust Standard Error:                             OLS Regression Results                            
Dep. Variable:           ln_race_time   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                     2437.
Date:                Tue, 16 Feb 2021   Prob (F-statistic):               0.00
Time:                        22:48:48   Log-Likelihood:                 17850.
No. Observations:               24699   AIC:                        -3.569e+04
Df Residuals:                   24694   BIC:                        -3.565e+04
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     

In [19]:
# Hidden Tests, checking the coefficients and standard errors of ols_model2_1.

**2.** Vaporfly shoes are pretty expensive, with a retail price of $250. Younger runners, who we expect would have lower income levels, may find it too expensive to purchase. Furthermore, it has been documented that in endurance type races, such as marathons, older runners are actually faster than their younger peers. Answer the following questions based on this information. 

**2a.** Would you expect the effect of the omitted variable *age* on the outcome variable *ln_race_time* to be positive or negative? (1 pt)

**Note**: This question will be manually graded.

YOUR ANSWER HERE

Negative, because older runners would be able to afford vaporfly shoes and alos are actually faster than their younger peers. 

**2b.** If we were to regress the omitted variable *age* on the treatment variable *vaporfly*, would you expect the coefficient in front of the variable *vaporfly* to be positive or negative? (1 pt)

**Note**: This question will be manually graded.

In [20]:
#Ols model regress omitted variable 'age' on treatment variable 'vaporfly':
model = smf.ols('age~vaporfly', data = data_marathon).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    age   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1050.
Date:                Tue, 16 Feb 2021   Prob (F-statistic):          1.61e-225
Time:                        22:48:48   Log-Likelihood:                -81617.
No. Observations:               24699   AIC:                         1.632e+05
Df Residuals:                   24697   BIC:                         1.633e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     32.8231      0.060    550.757      0.0

YOUR ANSWER HERE

I expect it to be positive due to possible positive correlation between older runners and vaporfly shoes (older runners are likely to be able to afford $250 vaporfly shoes).
To test this out, I ran the ols regression model between age and vaporfly treatment variable and observed a positive coefficient for vaporfly variable from the model summary as shown above. 

**2c.** State the OVB (omitted variable bias) formula. Based on it, would you expect the omitted variable bias to be positive or negative? Explain. (2 pts) 

**Note**: This question will be manually graded.

In [21]:
#Now, let's get the regression coefficient of treatment variable 'vaporfly' on omitted variable 'age', 
#which was calculated earlier by model.summary is equal to 2.7171.  

In [22]:
#Hence, 
OVB = round(-0.0086*2.7171,3)
print(f"Omitted Variable Bias: {OVB}")

Omitted Variable Bias: -0.023


YOUR ANSWER HERE

Omitted Variable Bias (OVB) formula =  (Effect of the omitted variable on the outcome) x (Effect of included variable on the omitted variable)

Based on above formula, I expect omitted variable to be negative because based on the causal model using nearest neighbor matching, the average treatment effect of age on people wearning vaporfly or not on the race time was -0.037. This tells me that people wearning the vaporfly shoes took shorter race time than people not wearing it, hence the regression coefficient of age should be a negative value. 
Moreover, when we regressed omitted variable 'age' on treatment variable 'vaporfly', the regression coefficient of vaporfly was a positive value 2.7171. Hence, s per the formula above OBV will be negative.  

Below, from question 3, we can also test this out by using OLS regression model. 

First, we calculate the effect of omitted variable on the outcome: (i.e if had included age in our ols regression model, then what would be the regression coefficient of the omitted variable age). This came out to be -0.0086 from omitted_age summary below.  

Hence, OVB  = -0.0086 x 2.7171

OVB = -0.023, which is a negative bias as expected. 


**3.** Let’s confirm the omitted variable bias. Run the regression from Part 2, Question 1 but this time also control for runners’ age by including the additional control variable age. 

In [23]:
# YOUR CODE HERE
#Get regression coefficient of omitted variable 'age' on outcome variable race_time: 
omitted_age =  smf.ols("ln_race_time~age+vaporfly+male+seasoned+enthusiastic", data = data_marathon).fit()
omitted_age.summary()
#So, here we can see that the coefficient of omittted variable (age) on the outcome is -0.0086. 
#raise NotImplementedError()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           ln_race_time   R-squared:                       0.464
Model:                            OLS   Adj. R-squared:                  0.464
Method:                 Least Squares   F-statistic:                     4273.
Date:                Tue, 16 Feb 2021   Prob (F-statistic):               0.00
Time:                        22:48:48   Log-Likelihood:                 21081.
No. Observations:               24699   AIC:                        -4.215e+04
Df Residuals:                   24693   BIC:                        -4.210e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        9.9345      0.003   2868.727      0.000       9.928       9.941
age             -0.0086   9.95e-05    -85.930      0.000      -0.009      -0.008
vaporfly        -0.0426      0.001    -31.773      0.000      -0.045      -0.040
male            -0.1296      0.001    -96.756      0.000      -0.132      -0.127
seasoned        -0.0889      0.002    -48.192      0.000      -0.093      -0.085
enthusiastic    -0.0548      0.003    -18.125      0.000      -0.061      -0.049
==============================================================================
Omnibus:                      570.884   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              646.028
Skew:                          -0.341   Prob(JB):                    5.21e-141
Kurtosis:                       3.404   Cond. No.                         186.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Compare the two coefficients in front of the treatment variable vaporfly, what do you observe? Based on your observation, are the results consistent with your previous intuitive analysis of OVB? (2 pts) 

**Note**: This question will be manually graded.

In [24]:
#Estimate bias between short and long regression of the treatment variable 'vaporfly'
beta_short = -0.0658
beta_long = -0.0426

bias = round((beta_short - beta_long),3)

print(f"Bias between short and long regression of the treatment variable: {bias}")


Bias between short and long regression of the treatment variable: -0.023


YOUR ANSWER HERE

Yes, I noticed that the results are consistent with my previous intuitive analysis of OVB. This is because when I ran the short and long regression models without and with the omitted variable 'age' respectively , the difference found between the regression coefficients of the treatment variable 'vaporfly' was similar to OVB we had calculated previously by applying the formula. 

i.e

bias = -0.023 

OVB = -0.023 